In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys

sys.path.append('/home/alanzhou93_gmail_com/substorm-detection')
sys.path.append('/home/alanzhou93_gmail_com/RNN')
sys.path.append('/home/alanzhou93_gmail_com/\baseline')
sys.path.append('/home/alanzhou93_gmail_com/\data')

from RNN import rnn_models
import utils
from baseline import utils_linear
import plot_utils
from baseline import linear_models

Using TensorFlow backend.


In [2]:
folder = '/home/alanzhou93_gmail_com/substorm-detection/data'
data_fn = folder + '/2classes_data128_withsw_small.npz'
data = np.load(data_fn)
X = data['X']
y = data['y'][:, None]
SW = data['SW']
strength = data['strength']

print(np.shape(X))
print(np.shape(y))
print(np.shape(SW))
print(np.shape(strength))

(33424, 81, 128, 3)
(33424, 1)
(33443, 256, 6)
(33424,)


In [3]:
batch_size = 64

train_test_split = .2
train_val_split = .2
train, test = utils.split_data([X, SW, y, strength], train_test_split, random=False)
train, val = utils.split_data(train, train_val_split, random=True, batch_size=batch_size)
X_train, sw_train, y_train, strength_train = train
X_val, sw_val, y_val, strength_val = val
X_test, sw_test, y_test, strength_test = test

X_train, X_val, X_test = utils.rnn_format_x([X_train, X_val, X_test])
y_train, y_val, y_test = utils.rnn_format_y([y_train, y_val, y_test])

X_train, X_val, X_test = [X_train, sw_train], [X_val, sw_val], [X_test, sw_test]
y_train, y_val, y_test = [y_train, strength_train], [y_val, strength_val], [y_test, strength_test]

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

In [4]:
params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 64,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 1,
    'verbose': True,
    'time_output_weight': 1e6,
    'rnn_type': 'GRU',
    'output_type': 'time'
}

In [37]:
hist, model = rnn_models.train_functional_rnn_single(X_train, y_train, X_val, y_val, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (256, 256, 6)        0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (256, 128, 243)      0                                            
__________________________________________________________________________________________________
gru_23 (GRU)                    (256, 256, 64)       13632       input_16[0][0]                   
__________________________________________________________________________________________________
gru_21 (GRU)                    (256, 128, 64)       59136       input_15[0][0]                   
__________________________________________________________________________________________________
gru_24 (GR

## Hyperparameter Search: RNN type

In [5]:
hist_array = []
model_array = []

for rnn_hidden_units in [32, 64, 96, 128]:
    
    print(rnn_hidden_units)
    
    params = {
    'batch_size': batch_size,
    'rnn_hidden_units': rnn_hidden_units,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 10,
    'verbose': True,
    'time_output_weight': 1e6,
    'rnn_type': 'gru',
    'output_type': 'time'
    }
    
    hist, model = rnn_models.train_functional_rnn_single(X_train, y_train, X_val, y_val, params)
    
    print('Accuracy: %.2f' %hist.history['val_acc'][-1])
    hist_array.append(hist)
    model_array.append(model)
    

32
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (64, 256, 6)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (64, 128, 243)       0                                            
__________________________________________________________________________________________________
gru_3 (GRU)                     (64, 256, 32)        3744        input_2[0][0]                    
__________________________________________________________________________________________________
gru_1 (GRU)                     (64, 128, 32)        26496       input_1[0][0]                    
__________________________________

KeyboardInterrupt: 

In [ ]:
sns.set()
plt.figure(figsize = (10,10))


## Compare separate networks to a combined network

In [9]:
    params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 128,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 15,
    'verbose': True,
    'time_output_weight': 1e6,
    'rnn_type': 'gru',
    'output_type': 'time'
    }

In [ ]:
hist, model = rnn_models.train_functional_rnn_combined(X_train, y_train, X_val, y_val, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (256, 256, 6)        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (256, 128, 243)      0                                            
__________________________________________________________________________________________________
gru_7 (GRU)                     (256, 256, 128)      51840       input_2[0][0]                    
__________________________________________________________________________________________________
gru_5 (GRU)                     (256, 128, 128)      142848      input_1[0][0]                    
__________________________________________________________________________________________________
gru_8 (GRU

## Training on only magnetometer data

In [7]:
hist, model = rnn_models.train_basic_gru(X_train[0], y_train[0], X_val[0], y_val[0], params)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 21248 samples, validate on 5120 samples
Epoch 1/15
21248/21248 [==============================] - 46s 2ms/step - loss: 0.6522 - acc: 0.6074 - true_positive: 0.5622 - false_positive: 0.3475 - val_loss: 0.6187 - val_acc: 0.6770 - val_true_positive: 0.6889 - val_false_positive: 0.3359
Epoch 2/15
21248/21248 [==============================] - 39s 2ms/step - loss: 0.5962 - acc: 0.6919 - true_positive: 0.6810 - false_positive: 0.2967 - val_loss: 0.5861 - val_acc: 0.7027 - val_true_positive: 0.7394 - val_false_positive: 0.3350
Epoch 3/15
21248/21248 [==============================] - 39s 2ms/step - loss: 0.5683 - acc: 0.7191 - true_positive: 0.7118 - false_positive: 0.2746 - val_loss: 0.5583 - val_acc: 0.7275 - val_true_positive: 0.6994 - val_false_positive: 0.2451
Epoch 4/15
21248/21248 [==============================] - 39s 2ms/step - loss: 0.5421 - acc: 0.7404 - 

## Training on only solar wind data

In [8]:
hist, model = rnn_models.train_basic_gru(X_train[1], y_train[0], X_val[1], y_val[0], params)

Train on 21248 samples, validate on 5120 samples
Epoch 1/15
21248/21248 [==============================] - 67s 3ms/step - loss: 0.6536 - acc: 0.6115 - true_positive: 0.5895 - false_positive: 0.3710 - val_loss: 0.6033 - val_acc: 0.6818 - val_true_positive: 0.7315 - val_false_positive: 0.3666
Epoch 2/15
21248/21248 [==============================] - 67s 3ms/step - loss: 0.6057 - acc: 0.6793 - true_positive: 0.6648 - false_positive: 0.3049 - val_loss: 0.5902 - val_acc: 0.7016 - val_true_positive: 0.6411 - val_false_positive: 0.2374
Epoch 3/15
21248/21248 [==============================] - 66s 3ms/step - loss: 0.6009 - acc: 0.6820 - true_positive: 0.6599 - false_positive: 0.2952 - val_loss: 0.5837 - val_acc: 0.6986 - val_true_positive: 0.6689 - val_false_positive: 0.2704
Epoch 4/15
21248/21248 [==============================] - 66s 3ms/step - loss: 0.5976 - acc: 0.6851 - true_positive: 0.6568 - false_positive: 0.2859 - val_loss: 0.5870 - val_acc: 0.6969 - val_true_positive: 0.6759 - val_fa